# Dataset 1: Butterfly & Moths Image Classification 100 species

## (0) Download Butterfly 

In [14]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species

 99%|███████████████████████████████████████▌| 395M/399M [00:11<00:00, 44.1MB/s]
100%|████████████████████████████████████████| 399M/399M [00:11<00:00, 36.1MB/s]


In [1]:
import zipfile
with zipfile.ZipFile('butterfly-images40-species.zip', 'r') as zip_ref:
    zip_ref.extractall('data/butterfly')

## (1) Process Butterfly Data

In [1]:
from utils.data import create_butterfly_dataset
trainset, testset, normal_mapping, reverse_mapping, sample_img_dataset = create_butterfly_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

## (2) Train & Save Classifier

In [ ]:
from model.butterfly_classifier import DenseNet121
from utils.base import train_classifier
from torch.utils.data import DataLoader
from torch import nn
import torch

# Params
batch_size=100
lr = 0.0001
device='cuda'

# Data loader
train_loader = DataLoader(dataset=trainset, batch_size=batch_size)
test_loader = DataLoader(dataset=testset, batch_size=batch_size)

# Model
model = DenseNet121(num_classes=len(normal_mapping)).to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

# load states
# model.load_state_dict(torch.load('./model/states/butterfly_classifier.pth'))

In [ ]:
from tqdm import trange

train_hist = list()
test_hist = list()
for epoch in trange(1, 5 + 1):
    train_loss, test_loss = train_classifier(model, train_loader, test_loader, optimizer, criterion, epoch)
    train_hist.append(train_loss)
    test_hist.append(test_loss)
    print('Epoch {}: Train: {}, Test: {}'.format(epoch, train_loss, test_loss))
    
torch.save(model.state_dict(), 'butterfly_classifier.pth')

## (3) Evaluate Classifier

Our classifier achieves an accuracy of 96.0% on the test set.

In [ ]:
from torch.utils.data import DataLoader
from utils.base import eval_accuracy


acc = eval_accuracy(model, testset,  batch_size=100)
print('Accuracy on test set is {}'.format(acc))

# Dataset 2: Tiny ImageNet

## (0) Download ImageNet 

In [ ]:
!kaggle datasets download -d akash2sharma/tiny-imagenet

In [ ]:
import zipfile
with zipfile.ZipFile('tiny-imagenet.zip', 'r') as zip_ref:
    zip_ref.extractall('data/tinyimagenet')

## (1) Process ImageNet Data

In [4]:
from utils.data import create_imagenet_dataset
trainset, testset, normal_mapping, reverse_mapping, sample_img_dataset = create_imagenet_dataset(img_reshape=(3, 224, 224), num_classes = 20)
assert len(trainset) == 8000, 'Size of train set not match'
assert len(testset) == 2000, 'Size of test set not match'

## (2) Train & Save Classifier

In [28]:
from model.imagenet_classifier import InceptionV3
from utils.base import train_classifier_imagenet

# Parameters
batch_size=100
lr = 0.0001

# Data Loader
train_loader = DataLoader(dataset=trainset, batch_size=batch_size)
test_loader = DataLoader(dataset=testset, batch_size=batch_size)

# model, loss, optimizer, and scheduler
model = InceptionV3(num_classes=20).to('cuda')
model = model.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

model.safetensors:   0%|          | 0.00/95.5M [00:00<?, ?B/s]

C:\Users\jinga\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jinga\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [29]:
from tqdm import trange

train_hist = list()
test_hist = list()
for epoch in trange(1, 5 + 1):
    train_loss, test_loss = train_classifier(model, train_loader, test_loader, optimizer, criterion, epoch)
    train_hist.append(train_loss)
    test_hist.append(test_loss)
    print('Epoch {}: Train: {}, Test: {}'.format(epoch, train_loss, test_loss))
    
torch.save(model.state_dict(), 'imagenetclassifier.pth')

 20%|████████████████▌                                                                  | 1/5 [03:53<15:33, 233.31s/it]

Epoch 1: Train: 1.3993307277560234, Test: 0.6634422808885574


 40%|█████████████████████████████████▏                                                 | 2/5 [07:49<11:45, 235.12s/it]

Epoch 2: Train: 0.333159423712641, Test: 0.6131627306342124


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [11:40<07:46, 233.32s/it]

Epoch 3: Train: 0.06279878542991355, Test: 0.6467071786522866


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [15:34<03:53, 233.51s/it]

Epoch 4: Train: 0.012380183630739339, Test: 0.6652144812047481


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [19:26<00:00, 233.21s/it]

Epoch 5: Train: 0.005448856894508936, Test: 0.6708217523992062


## (3) Evaluate Classifier

In [30]:
acc = eval_accuracy(model, testset,  batch_size=100)
print('Accuracy on test set is {}'.format(acc))

Accuracy on test set is 0.8355000615119934
